## Demo for SDK (latest)

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [get `job_source_uri`](#get-job_source_uri)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

In [20]:
import sys
sys.path.insert(0, '..')

import os
import time
import dotenv
dotenv.load_dotenv()
from swan import SwanAPI , MCSAPI

dev_url = "https://swanhub-cali.swanchain.io"
# dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
# dev
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)
# prod
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"))

# api_key = os.getenv("MCS_API_KEY")
# mcs_api = MCSAPI(api_key)

from swan.contract.swan_contract import SwanContract

contract = SwanContract(os.getenv('PK'), swan_api.contract_info)

In [2]:
r = swan_api.contract_info
print(r)

{'client_contract_address': '0x9c5397F804f6663326151c81bBD82bb1451059E8', 'payment_contract_address': '0xB48c5D1c025655BA79Ac4E10C0F19523dB97c816', 'rpc_url': 'https://rpc-atom-internal.swanchain.io', 'swan_token_contract_address': '0x91B25A65b295F0405552A4bbB77879ab5e38166c'}


### Show available hardware information

In [3]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['-', 'North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['-', 'North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['-', 'North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'reigion': ['-', 'North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['-'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'description': 'Nvidia 3090 · 8 vCPU · 16 GiB

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [4]:
cfg_name = 'C1ae.medium' #"G1ae.medium"
hardware = [hardware for hardware in hardwares if hardware.name == cfg_name][0]
print(f"{hardware.name=}, {hardware.id=}, {hardware.region}")

hardware_id = hardware.id
print(f"The chosen {hardware_id=}")

hardware.name='C1ae.medium', hardware.id=1, ['-', 'North Carolina-US']
The chosen hardware_id=1


In [5]:
hardware_id

1

### Get `job_source_uri`

We can use the `get_source_uri` interface to get a `job_source_uri` for creating task.

In [6]:

job_source_uri = swan_api.get_source_uri(
    repo_uri='https://github.com/alphaflows/tetris-docker-image.git',
    hardware_id=hardware_id,
    wallet_address=os.getenv('WALLET')
)

repo_uri='https://github.com/alphaflows/tetris-docker-image.git', hardware_id=1, wallet_address='0xaA5812Fb31fAA6C073285acD4cB185dDbeBDC224'repo_owner=None, repo_name=None, None


In [8]:
if job_source_uri and job_source_uri.get('data'):
    job_source_uri = job_source_uri['data']['job_source_uri']

In [10]:
job_source_uri

'https://plutotest.acl.swanipfs.com/ipfs/Qmf5iZCmTNaT2vf279XuPywa8W3rLi5vXbtPBJDzniYxvh'

### Estimate Payment amount

In [11]:
duration_hour = 1 # hour
amount = contract.estimate_payment(hardware_id, duration_hour)
print(amount)

1000000000000000000


### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [13]:
import json

duration=3600*duration_hour

result = swan_api.create_task(
    cfg_name=cfg_name, 
    region='North Carolina-US', 
    start_in=5, 
    duration=duration, 
    job_source_uri=job_source_uri,#repo.source_uri, 
    paid=contract._wei_to_swan(amount),
    wallet_address=os.getenv('WALLET'),
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

{
  "data": {
    "task": {
      "created_at": "1714170299",
      "end_at": "1714173896",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "initialized",
      "task_detail_cid": "https://plutotest.acl.swanipfs.com/ipfs/QmQJQbbdAN2aonAoURP6p6MRQhvUSqnaVZWfT1eXj9gTQP",
      "tx_hash": null,
      "updated_at": "1714170299",
      "uuid": "eab40857-2601-45c6-a2d1-b22b29ea8692"
    }
  },
  "message": "Task_uuid initialized.",
  "status": "success"
}


### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [14]:
tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
print(tx_hash)

0xc562abf4bccc14f9338d189ecc6d90797bfef01e882fb3f4db6949e088e5e447


### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [15]:
if result_validation := swan_api.validate_payment(
    tx_hash=tx_hash,
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

{'tx_hash': '0xc562abf4bccc14f9338d189ecc6d90797bfef01e882fb3f4db6949e088e5e447', 'task_uuid': 'eab40857-2601-45c6-a2d1-b22b29ea8692'}
{
  "data": {
    "task": {
      "created_at": "1714170299",
      "end_at": "1714173896",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "created",
      "task_detail_cid": "https://plutotest.acl.swanipfs.com/ipfs/QmQJQbbdAN2aonAoURP6p6MRQhvUSqnaVZWfT1eXj9gTQP",
      "tx_hash": null,
      "updated_at": "1714170321",
      "uuid": "eab40857-2601-45c6-a2d1-b22b29ea8692"
    }
  },
  "message": "Task payment validated successfully.",
  "status": "success"
}


### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [17]:
# Check task info
info = swan_api.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "allowed_nodes": null,
        "autobid": 1,
        "city": "Raleigh",
        "country": null,
        "created_at": "1709866791",
        "deleted_at": null,
        "last_active_at": null,
        "lat": 35.8639,
        "lon": -78.535,
        "multi_address": "/ip4/provider.cp.filezoo.com.cn/tcp/9086",
        "name": "ubi-test-01",
        "node_id": "04f76ddea946cd44634fc68d09b12d8a763d7616069d68ddd3587ae5e0deafcca5777d09fad7bd7c84a143d153800df17534916dc1708ee5dd8d48f6e938ab4eb6",
        "online": true,
        "public_address": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "region": "North Carolina-US",
        "score": 100,
        "status": "Active",
        "updated_at": "1714170114"
      }
    ],
    "jobs": [
      {
        "bidder_id": "04f76ddea946cd44634fc68d09b12d8a763d7616069d68ddd3587ae5e0deafcca5777d09fad7bd7c84a143d153800df17534916dc1708ee5dd8d48f6e938ab4eb6",
        "build_log": "wss://log.cp.

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [19]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://a2kwf1mzff.cp.filezoo.com.cn']


In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)
